In [1]:
import os
import torch
from PIL import Image
import numpy as np
import cv2

In [2]:
from transformers import AutoModelForVision2Seq

In [3]:
import robosuite as suite
from robosuite.controllers import load_controller_config
from robosuite.wrappers import Wrapper

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /anvil/projects/x-cis250308/miniconda/lib/python3.13/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)


In [3]:
from transformers import AutoProcessor

2025-11-18 10:06:58.592411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
device = torch.device("cuda")

In [21]:
processor = AutoProcessor.from_pretrained(
    "openvla/openvla-7b",
    trust_remote_code=True,
    cache_dir = "./vla_cache"
)

In [9]:
# vla = AutoModelForVision2Seq.from_pretrained(
#     "openvla/openvla-7b",
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     low_cpu_mem_usage = True,
#     cache_dir = "./vla_cache"
# )

In [22]:
vla = AutoModelForVision2Seq.from_pretrained(
    "./vla_cache",
    local_files_only = True,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage = True,
    trust_remote_code=True,
)

Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.46.3` and `tokenizers==0.20.3`; there might be inference-time regressions due to dependency changes. If in doubt, pleaseuse the above versions.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
vla = vla.to(device)

In [18]:
env = suite.make(
    "Lift",                     # simple manipulation task
    robots="Panda",             # Franka Panda 7-DoF arm
    has_renderer=True,          # <-- GUI window ON
    has_offscreen_renderer=True,
    use_camera_obs=True,        # return camera obs
    camera_names=["frontview", "birdview"], # one camera is enough
    camera_heights=224,
    camera_widths=224,
    render_camera="frontview",  # show this in GUI
    control_freq=20,            # 20 Hz control
)

In [19]:
env = Wrapper(env)

In [24]:
prompt = "Touch the cube"

In [25]:
obs = env.reset()

In [ ]:
# from IPython.display import display
# frame = obs["frontview_image"]
# prompt = "take the cube"
# img = Image.fromarray(frame).transpose(method=Image.FLIP_TOP_BOTTOM)
# display(img)
# inps = processor(text = [prompt], images = [img]).to(device=device, dtype=torch.bfloat16)
# with torch.no_grad():
#     action = vla.predict_action(
#         **inps,
#         unnorm_key = "bridge_orig",
#         do_sample = False
#     )
# action = action.tolist()
# print(action)

In [17]:
for step in range(200):
    frame1 = obs["frontview_image"]
    image1 = Image.fromarray(frame1).transpose(method=Image.FLIP_TOP_BOTTOM)
    # frame2 = obs["birdview_image"]
    # image2 = Image.fromarray(frame2)
    env.render()
    inputs = processor(text = [prompt], images = [image1]).to(device=device, dtype=torch.bfloat16)
    with torch.no_grad():
        action = vla.predict_action(
            **inputs,
            unnorm_key = 'bridge_orig',
            do_sample=False
        )
    action = action.tolist()
    action.append(action[-1])
    obs, reward, done, info = env.step(action)

AttributeError: 'NoneType' object has no attribute 'render'

In [16]:
env.close()